Basic Sina Usage
==============

Welcome to Sina! This tutorial will guide you through its core functionality. If you want to jump right into exploring data (and/or don't expect to interact with the API directly), you may want to jump to the [visualization tutorial](vis_usage.ipynb).

## Core Concepts


Sina's job is making managing simulation data easier, so we try to keep the process simple:

- Sina's "unit of data"--the thing you store, retrieve, and ask questions about--is a `Record`. To be brief, **one simulation run is one Record** (but the word Record is more source-agnostic). Essentially, a `Record` is a set of inputs and their corresponding outputs.

- **You can work with** `Records` **individually.** Sina gives you a Python object with some nice quality of life methods for post-processing, but under the hood, that Python object is functionally the same JSON your simulation code may be dumping out (they tend to be named `*_sina.json`). Another way of putting it is that **Sina turns your simulation runs into simple Python objects that can be manipulated according to your post-processing needs**.

- **Much of Sina's API is dedicated to looking at** `Records` **in aggregate**. Say you've done dozens/hundreds/millions of runs and you want to pick the interesting ones. Sina's API contains a series of "questions" you can ask. Which runs had the highest `etot`? Which ended within 300 `cycle`s? Which had `laser_package` turned on and `debug` off?

- **A big part of Sina is using these capabilities in tandem**. Use the "questioning" part of the API to narrow down which runs you want to look at, then use the returned objects to take a closer look. For example, you get all the runs where `max_temp` went above some number, then use the objects to re-launch with slightly different settings.

## Building Our "Simulation Runs"

To keep this tutorial portable and stop it from cluttering up your filesystem, we'll avoid working with real simulation data, instead making a couple "runs" from scratch. To start with, we'll make something in the Sina JSON schema, to make it easier to compare it to what a physics code might be putting out:

In [ ]:
import json
import random

import sina
from sina.model import Record, generate_record_from_json
from sina.utils import DataRange, has_all, has_any, any_in, all_in

simple_record_structure = {
    # The id uniquely identifies the run. It's often something like timestamp+UUID
    "id": "my_example_breakfast_simulation",

    # The type helps us categorize runs, in case there's several types of "thing" in the datastore.
    # For example, you might have runs plus information from the msubs that submitted them ("type": "msub")
    "type": "breakfast_sim_run",

    # Data is the meat and potatoes of the run. These are your inputs, outputs, metadata etc. that allow you to
    # select a run from the pile. For example, thanks to this data block, we'd be able to pull this record back if
    # we asked for all runs with an omelette_count > 1.
    "data": {"egg_count": {"value": 10},
             "omelette_count": {"value": 3},
             # In addition to a value, entries can have units and tags...we'll hold off for now
             "flavor": {"value": "tasty!"}}
    # There's other sections we can have in a record (ex: curve_sets), but we'll keep it simple for now!
}

simple_record = generate_record_from_json(simple_record_structure)

print("Created a basic record!")

Making Records Programatically
--------------------------------------

We'll speed things up by generating the rest with Python. These will be a bit more fleshed out, and a bit more realistic--we skip the notion of fiddling with JSON entirely here, making Sina's `Record`s directly. This is a great way to do things if your code is already in Python. Just add in a Record wherever you're defining your inputs and call `add_data` as appropriate.

We first create a new Record object (covering the `id` and `type` chunks in the cell above), then assign it some `data`. Realistically, data will be provided by whatever's launching your runs, but here we generate some at random.

You could dump any of these Record objects to a JSON file if you like, and they'd be formatted just like what's output by a simulation run--and that simulation run would be formatted just like the output of some other code using Sina. A big part of Sina's power is in the notion of that common file format.

We're just making the objects themselves for now. We'll access one to make sure they look alright, and in the next cell, we'll prep them for querying!

In [ ]:
possible_maintainers = ["John Doe", "Jane Doe", "Gary Stu", "Ann Bob"]


def generate_run(idx):
    # Our sample "code runs" are mostly random data. We'll generate some inputs...
    record = Record(id="rec_{}".format(idx), type="foo_prod_run")
    record.add_data('initial_density', random.randint(10, 1000) / 10.0, units='g/cm^3')
    record.add_data('num_procs', random.randint(1, 4))
    record.add_data('maintainer', random.choice(possible_maintainers), tags=["personnel"])
    # Pretend we ran a simulation in here...and add the outputs and artifacts.
    if random.randint(1, 6) == 6:
        record.add_file("{}_log.txt".format(idx))
    record.add_data('final_volume', random.random() * 100)
    return record


print("Defined a function for generating more records!")

test_rec = generate_run(0)
print("Accessing randomly-generated test record {}. Its initial_density is: {}.\nRe-running this cell will reroll this value."
      .format(test_rec.id,
              test_rec.data["initial_density"]["value"]))

## Asking Questions about Our Runs

In order to question all our simulation runs at once (ex: which runs had John Doe as a maintainer?), we'll need to let Sina know what runs we want to question. We'll add our simulations to a `datastore`, a generic term for Sina taking data from the JSONs/Records and organizing it in a way that makes asking cross-run questions efficient. This can be done from the command line too, if you prefer (use `sina -h` to access the CLI help or, if you're accessing this through the docs, see [here](../cli_basics.html)).

First we connect to an in-memory `datastore`...

In [ ]:
# The default (read: without an argument) behavior of sina.connect() is to connect to an in-memory SQLite database.
# These are temporary, wiped from memory once they're closed. Good for tutorials, not so good for data storage!
# If you'd like to create a proper file, just provide the filename as an arg: sina.connect("my_db.sqlite")
# You can also pass the URL to a database such as MySQL or MariaDB.
ds = sina.connect()
print("Connection is ready!")

...and now we insert our records! We'll first insert our simple record, then use the function we made to generate several more. `num_to_generate` is largely arbitrary, my default of 500 is intended to give a reasonable number of results for the rest of this tutorial.

In [ ]:
ds.records.insert(simple_record)
print("The simple Record has been inserted into the datastore")

num_to_generate = 500

for record_idx in range(0, num_to_generate):
    ds.records.insert(generate_run(record_idx))

print("{} randomly-generated Records have been inserted into the datastore.".format(num_to_generate))

### Our first question: finding Records based on data

Let's ask a question of all our runs. How many had John Doe as a maintainer? For this, we'll use Sina's `find_with_data()` which, as you might guess, allows us to find runs based on the content of that `data` section we mentioned back in the first code cell.

One thing you should notice--we call `list()` on what's returned, as Sina returns generators. Useful for memory efficiency on large queries!

In [ ]:
maintainer_to_query = "John Doe"

print("Found {} runs with {} listed as their maintainer".format(
    len(list(ds.records.find_with_data(maintainer=maintainer_to_query))),
    maintainer_to_query))

### Our second question: finding Records based on more complicated criteria

What John's really interested in is runs where the `final_volume` seemed low. Anything with a volume of 6 or below is low for our fake simulation, so we'll get those.

John did a few diagnostic runs that he'd like to exclude from the set, so we'll also make sure we only select Records with a type of `foo_prod_run`.

This time, we'll use `find()`! It's Sina's most powerful and flexible query, blending together a number of others (including `find_with_data()`) into one efficient mega-query. Here, it'll let us query on a Record's data and its type at the same time.

We'll also make our first use of `DataRange`s. These are exactly what they sound like--objects that let us specify that some quantity should fall within a range. DataRanges allow you to configure the min and max behavior (inclusive vs. exclusive). By default, it's min inclusive, max exclusive, same as `range()`.

One last note! `ids_only=True`: recall that simulation runs are stored into Python objects. Sina's queries are happy to give you those objects directly, but sometimes we only want the ids associated with them. Using `ids_only=True` where appropriate can really speed up your queries.

In [ ]:
# Let's throw an extra record in there to be sure we have at least one match!
template_rec = Record(id="john_latest", type="foo_prod_run")
template_rec.add_data("final_volume", 6)
template_rec.add_data("initial_density", 6, units="cm^3")
template_rec.add_data("maintainer", "John Doe")

ds.records.insert(template_rec)

# John's diagnostic run! "Coincidentally" looks very similar to the above. We don't want this one, though!
# Since we're changing the id, the datastore will see this as an entirely new run.
template_rec.id = "dont_fetch_me"
template_rec.type = "foo_diagnostic"
ds.records.insert(template_rec)

# Now we prepare a dictionary of criteria. This is equivalent to:
# find_with_data(maintainer="John Doe", final_volume=DataRange(max=6, max_inclusive=True))
# However, find() is more flexible than find_with_data()--we pass data criteria as a dict so we can combine them
# with other things (here, record types). Combined queries tend to be more efficient!
target_data = {"maintainer": "John Doe",
               "final_volume": DataRange(max=6, max_inclusive=True)}

john_low_volume = ds.records.find(data=target_data, types="foo_prod_run", ids_only=True)

print("John Doe's low-volume production runs: {}".format(', '.join(john_low_volume)))

### An aside: run management


Sometimes we want to alter the runs in our datastore. For example, we don't really want that `foo_diagnostic` kicking around in our "production" database. We can delete (and update!) things within Sina, but I ALWAYS recommend keeping backups. If you want to ensure you don't do anything by accident, when you connect(), do `connect(read_only=True)`. We didn't do that there, so we can delete away.

In [ ]:
ds.records.delete("dont_fetch_me")

Really getting into the weeds here, but if you're worried about someone accidentally wiping out your data, you can lock it down using MySQL users. Setting up a MySQL database is out of scope for these tutorials, but LC users should visit [LaunchIT](https://launchit.llnl.gov) for up-to-date documentation.

### Special queries

John's still curious about how low the `final_volume` can be, regardless of who ran them. Sina was designed with these sorts of questions in mind--there are specialized query functions for performing common selections like "Records with the <number> lowest values for X" or "ALL records in the datastore", which can be put to even further uses with args (ex: you can get the total number of records with `len(list(ds.records.get_all(ids_only=True)))`). See the [datastore documentation](../generated_docs/sina.datastore.rst) for the full list!

In [ ]:
# John wants the 3 Records with the lowest volumes.
runs_with_lowest_volumes = ds.records.find_with_min("final_volume", 3)
print("The runs with the lowest volume:")
for run in runs_with_lowest_volumes:
    print("{}: {} (maintainer: {})".format(run.id,
                                           run.data["final_volume"]["value"],
                                           run.data["maintainer"]["value"]))

### Congratulations, that's the basic basics!

In order to serve diverse data and ask diverse questions, Sina has many more queries and capabilities included. While the rest of this tutorial is still "basic usage", not all sections will apply to all users, and the topics covered so far (building Records, inserting them, and querying them) form the backbone of Sina.

If you'd like to break away from this tutorial to try working with your own data, this is a natural point to do so. You may also want to check out the [visualization tutorial](vis_usage.ipynb), as the combination of that and the topics we've covered may be enough to get you started on a custom "dashboard" for your project.

Otherwise, we'll now move on to topics like combining queries, querying list data (ex: timeseries), handling data from libraries, associating sets of curves, and more!

### Combining queries

Not every query can be predicted, and we try to keep the main API simple. This is where Sina's Python nature comes in handy! Data can be handled and transformed, and queries can be cast to sets to combine them.

Seeing that he may not be the only one with this issue, John decides to investigate further. While they don't record initial mass, he knows he can find it from `density` and `volume` (mass is conserved throughout the simulation), and that an initial mass below 45 indicates that something strange has happened. So he first finds all runs where `(initial_density*final_volume)<45`, then figures which of those, if any, belong to him.

To save time in the future, John might want to add the `initial_mass` he calculated to the Record, then `update()` the datastore with it. See the [post-processing tutorial](post_processing.ipynb) for more details!

In [ ]:
# First we get the data we'll need
record_data = ds.records.get_data(["final_volume", "initial_density"])
low_mass_records = set()

# Then we go entry-by-entry, calculating the mass associated with each record id
# We'll use this to assemble a set of runs with mass < 45
for rec_id, data_dict in record_data.items():
    mass = data_dict["initial_density"]["value"] * data_dict["final_volume"]["value"]
    if mass < 45:
        low_mass_records.add(rec_id)
print("Low-mass runs: {}".format(low_mass_records))

# Now that we have our set of low mass runs, we'll intersect it with the set of John's runs
john_runs = list(ds.records.find_with_data(maintainer="John Doe"))
print("John's low-mass runs: {}".format(low_mass_records.intersection(john_runs)))

That done, keep in mind that Sina's `find()` allows you to combine multiple different queries, returning only Records that fulfill ALL your criteria. John's use of set operations allows him to do things not supported directly by Sina (like arbitrary calculations "within" a query), while `find()` is typically more efficient than set operations in terms of both performance and memory. Each has its uses!

For another `find()` example, Ann is doing some debugging and wants runs done on a certain number of processors, but only if they output a log.

In [ ]:
target_num_procs = 1

ann_runs = ds.records.find(data={"num_procs": target_num_procs},
                           types=["foo_prod_run"],
                           file_uri="%_log.txt",
                           ids_only=True)

print("Ann's target runs: {}".format(list(ann_runs)))

## List Data and Querying Them

Of course, Sina also supports storing and querying list-type data: timeseries, options activated, and nodes being a few examples. Note that, to maintain querying efficiency, a list can't have strings AND have scalars AND be queryable; only all-scalar or all-string lists can be part of a Record's data.

In [ ]:
# Records expressed as JSON strings. We expect records 1 and 3 to match our query.
record_1 = """{"id": "list_rec_1",
               "type": "list_rec",
               "data": {"options_active": {"value": ["quickrun", "verification", "code_test"]},
                        "velocity": {"value": [0.0, 0.0, 0.0, 0.0, 0.0]}},
               "user_defined": {"mixed": [1, 2, "upper"]}}"""
record_2 = """{"id": "list_rec_2",
               "type": "list_rec",
               "data": {"options_active": {"value": ["quickrun", "distributed"]},
                        "velocity": {"value": [0.0, -0.2, -3.1, -12.8, -22.5]}},
               "user_defined": {"mixed": [1, 2, "upper"],
                                "nested": ["spam", ["egg"]]}}"""
record_3 = """{"id": "list_rec_3",
               "type": "list_rec",
               "data": {"options_active": {"value": ["code_test", "quickrun"]},
                        "velocity": {"value": [0.0, 1.0, 2.0, 3.0, 4.1]}},
               "user_defined": {"nested": ["spam", ["egg"]],
                                "bool_dict": {"my_key": [true, false]}}}"""

for record in (record_1, record_2, record_3):
    ds.records.insert(generate_record_from_json(json.loads(record)))
print("3 list-containing Records have been inserted into the database.\n")

# Find all the Records that have both "quickrun" and "code_test" in their options_active
quicktest = ds.records.find_with_data(options_active=has_all("quickrun", "code_test"))

# Get those Records and print their id, value for options_active, and the contents of their user_defined.\n",
print("Records whose traits include 'quickrun' and 'code_test':\n")
for id in quicktest:
    record = ds.records.get(id)
    print("{} traits: {} | user_defined: {}".format(id,
                                                    ', '.join(record['data']['options_active']['value']),
                                                    str(record['user_defined'])))

### Further List Queries

There are a few additional ways to retrieve Records based on their list data. A `has_any()` query works on string lists, and will retrieve any Record that contains *at least* one of its args. An `all_in()` query retrieves Records where all members of a scalar list are in some range, while `any_in()` retrieves Records where one or more scalars are in the range. **Scalar ranges are assumed to be continuous.**

It's important to note that, for these types of list query, order and count don't matter. If `["quickrun", "code_test"]` would match, so would `["code_test", "quickrun", "quickrun"]`.

In [ ]:
match_has_any = list(ds.records.find_with_data(options_active=has_any("quickrun", "code_test")))
print("Records whose traits include 'quickrun' and/or 'code_test': {}".format(', '.join(match_has_any)))


match_all_in = list(ds.records.find_with_data(velocity=all_in(DataRange(min=0, max=0, max_inclusive=True))))
print("Records where velocity never changed from zero: {}"
      .format(', '.join(match_all_in)))


match_any_in = list(ds.records.find_with_data(velocity=any_in(DataRange(min=0, min_inclusive=False))))
print("Records that had a velocity greater than zero at some point: {}"
      .format(', '.join(match_any_in)))

## Curve_Set Basics

Sina can also store *collections* of list data. These "curve sets" are useful for tracking relationships between curves for ex: plotting. They're not themselves used in many queries, but their list data are available through the aforementioned list queries. If you're familiar with ULTRA files and PyDV, curve_sets are our way of storing that form of data (and integrating with PyDV).

In [ ]:
curve_rec = Record("curve_rec", "curve_rec")
# Note that similar methods (ex: add_file()) exist for other
# types of record info. Raw JSON is used in this notebook
# for at-a-glance readability, but the utility methods
# are generally recommended for "real" code.
sample_curve_set = curve_rec.add_curve_set("sample_curves")
sample_curve_set.add_independent("time", [0, 1, 2])
sample_curve_set.add_dependent("amount", [12, 14, 7])

ds.records.insert(curve_rec)
rec_with_curve_id = ds.records.find_with_data(amount=any_in(DataRange(min=12)))
print('Records with an "amount" >= 12 at some point: {}'
      .format(list(rec_with_curve_id)))

## Library Data Basics

In the case of simulations and similar, where an experiment may be the result of code components working together, consider using Sina's `library_data`. This allows for the grouping of data by source *within* a Record without fear of name collision with other groups, which is, true to name, most associated with libraries within a simulation.

Here we have a record, `library_rec`, with a library named `outer_lib` that wants to store its runtime, and an additional library, `inner_lib`, that wants to store ITS runtime. Of course, `library_rec` ALSO has a runtime. Using `library_data`, we can ensure that all three can be accessed.

In [ ]:
library_data_rec = """{"id": "library_rec",
                       "type": "library_rec",
                       "data": {"runtime": {"value": 12}},
                       "library_data": {
                           "outer_lib": {
                               "data": {"runtime": {"value": 10}},
                               "library_data": {
                                   "inner_lib": {
                                       "data": {"runtime": {"value": 4}}}}}}}"""
ds.records.insert(generate_record_from_json(json.loads(library_data_rec)))

runtimes = ds.records.get_data(["runtime", "outer_lib/inner_lib/runtime"])
print('Runtimes of the record itself, plus that of "inner_lib":')
for key in runtimes["library_rec"].keys():
    print(key, runtimes["library_rec"][key]["value"])

## Releasing Resources

When we're all done, it's important to release database resources. Failure to close connections can result in the server keeping additional resources open, resulting in performance issues.

In [ ]:
ds.close()

In order to not forget to close them, we can use our datastores in context managers.

In [ ]:
with sina.connect() as ds:
    # Since we closed the connection above, sqlite dropped the database and we created a new one.
    # We need to re-populate it.
    # This only happens with in-memory databases, of course! You're probably not using one.
    for record in (record_1, record_2, record_3):
        # We'll re-insert the records from List Data and Querying Them
        ds.records.insert(generate_record_from_json(json.loads(record)))
    print(list(ds.records.find_with_data(velocity=any_in(DataRange(min=-10, max=-5)))))
# Once we exit the context, since it's an in-memory db, it's once again dropped.

That's all for this tutorial. Thanks for your time, and good luck!